In [1]:
# Import necessary modules
import sys
import os
import torch 
# Set root folder to project root
os.chdir(os.path.dirname(os.getcwd()))

# Add root folder to path
sys.path.append(os.getcwd())

from src.utils.config import Hyperparameters
from src.utils.model_loading import load_model
from src.utils.maze_loading import load_mazes
from src.utils.testing import is_correct
from src.utils.analysis import plot_mazes

In [ ]:
# Load model
dt_net_original = load_model('dt_net')
dt_net = load_model('models/dt_net/2025-03-12_03:43:44/best.pth')
it_net = load_model('models/it_net/2025-03-26_23:59:57/best.pth')
model = dt_net

# Load mazes
hyperparams = Hyperparameters()
hyperparams.iters = 30
hyperparams.num_mazes = 10
hyperparams.percolation = 0.0
hyperparams.maze_size = 9
inputs, solutions = load_mazes(hyperparams)

# Predict
# predictions = model.predict(inputs, iters=hyperparams.iters)
with torch.no_grad():
    model.eval()
    latents = model.input_to_latent(inputs)
    print(f'{latents.mean().item():.4f}')
    latents = model.latent_forward(latents, inputs, iters=hyperparams.iters)
    print(f'{latents.mean().item():.4f}')
    #latents = model.latent_forward_layer(torch.cat([latents, inputs], dim=1))
    outputs = model.latent_to_output(latents)
    print(f'{outputs.mean().item():.4f}')
    predictions = model.output_to_prediction(outputs, inputs)
    print(f'{predictions.mean().item():.4f}')

# Evaluate predictions
corrects = is_correct(inputs, predictions, solutions)
print(f'{corrects.sum()} out of {len(corrects)} predictions are correct.')
print(f'outputs/visuals/mazes/predictions_{model.name}_size-{hyperparams.maze_size}_iters-{hyperparams.iters}')
# Plot results
plot_mazes(
    inputs,
    solutions,
    predictions,
    file_name=f'outputs/visuals/mazes/predictions_{model.name}_size-{hyperparams.maze_size}_iters-{hyperparams.iters}'
)

2025-03-29 13:25:00,859 - src.utils.model_loading - INFO - Loaded dt_net to cuda:0
2025-03-29 13:25:00,865 - src.utils.model_loading - INFO - Loaded models/dt_net/2025-03-12_03:43:44/best.pth to cuda:0
2025-03-29 13:25:00,875 - src.utils.model_loading - INFO - Loaded models/it_net/2025-03-26_23:59:57/best.pth to cuda:0
2025-03-29 13:25:00,876 - src.utils.maze_loading - INFO - Attempting 10 mazes to generate 10 mazes with size: 9, percolation: 0.0, and deadend_start: True
2025-03-29 13:25:00,941 - src.utils.maze_loading - INFO - Loaded 10 mazes with size: 9, percolation: 0.0, and deadend_start: True


0 out of 10 predictions are correct.
outputs/visuals/mazes/predictions_dt_net_size-9_iters-30
